In [42]:
import json
import pandas as pd
import sqlite3
import requests

In [43]:
conn = sqlite3.connect('Data/finalproject.db')

In [3]:
data = json.loads(open('Data/locations.json').read())

In [4]:
data2 = [record for record in data if type(record) == dict]

In [5]:
len(data2)

4948

In [6]:
data2 = data2[::2]

In [7]:
df = pd.DataFrame(data2)

In [8]:
df = df.drop_duplicates()

In [9]:
df.to_sql(name = 'gym_locations', con = conn, if_exists = 'replace')

2469

In [92]:
#response = requests.get('https://chronicdata.cdc.gov/resource/hn4x-zwk7.json?question=Percent of adults aged 18 years and older who have an overweight classification&yearstart=2021')
endpoint = 'https://chronicdata.cdc.gov/resource/hn4x-zwk7.json?question=Percent of adults aged 18 years and older who have an overweight classification&yearstart=2021'
query = {'$limit':89000,'$offset':0,'$$app_token': 'mG6piKQ1eLQSagm22PyMvQj3n'}

results = []

while True:
    response = requests.get(endpoint,params = query)
    if len(response.json()) == 0:
        break
    results.extend(response.json())
    
    query['$offset'] += len(response.json())


In [93]:
health = pd.DataFrame(results)

In [25]:
health.locationabbr.value_counts().shape

(55,)

In [32]:
health.sort_values('locationabbr').stratificationcategoryid1.value_counts()

RACE     440
INC      385
AGEYR    330
EDU      220
GEN      110
OVR       55
Name: stratificationcategoryid1, dtype: int64

In [94]:
health = health.drop(columns = 'geolocation')

In [40]:
health.to_sql(name = 'health', con = conn, if_exists = 'replace')

1540

In [37]:
health.iloc[0,15]

{'latitude': '43.65595011300047',
 'longitude': '-71.50036091999965',
 'human_address': '{"address": "", "city": "", "state": "", "zip": ""}'}

In [44]:
response = requests.get('https://api.census.gov/data/2019/pep/charagegroups?get=NAME,POP&for=state:*')

In [45]:
response.json()

[['NAME', 'POP', 'state'],
 ['Mississippi', '2976149', '28'],
 ['Missouri', '6137428', '29'],
 ['Montana', '1068778', '30'],
 ['Nebraska', '1934408', '31'],
 ['Nevada', '3080156', '32'],
 ['New Hampshire', '1359711', '33'],
 ['New Jersey', '8882190', '34'],
 ['New Mexico', '2096829', '35'],
 ['New York', '19453561', '36'],
 ['North Carolina', '10488084', '37'],
 ['North Dakota', '762062', '38'],
 ['Ohio', '11689100', '39'],
 ['Oklahoma', '3956971', '40'],
 ['Oregon', '4217737', '41'],
 ['Pennsylvania', '12801989', '42'],
 ['Rhode Island', '1059361', '44'],
 ['South Carolina', '5148714', '45'],
 ['South Dakota', '884659', '46'],
 ['Tennessee', '6829174', '47'],
 ['Texas', '28995881', '48'],
 ['Vermont', '623989', '50'],
 ['Utah', '3205958', '49'],
 ['Virginia', '8535519', '51'],
 ['Washington', '7614893', '53'],
 ['West Virginia', '1792147', '54'],
 ['Wisconsin', '5822434', '55'],
 ['Wyoming', '578759', '56'],
 ['Puerto Rico', '3193694', '72'],
 ['Alabama', '4903185', '01'],
 ['Alaska',

In [50]:
population = pd.DataFrame(response.json()[1:],columns = ['Name','Pop','State'])

In [51]:
population

,Name,Pop,State
0,Mississippi,2976149,28
1,Missouri,6137428,29
2,Montana,1068778,30
3,Nebraska,1934408,31
4,Nevada,3080156,32
5,New Hampshire,1359711,33
6,New Jersey,8882190,34
7,New Mexico,2096829,35
8,New York,19453561,36
9,North Carolina,10488084,37


In [53]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [56]:
population['abbr'] = population['Name'].map(us_state_to_abbrev)

In [57]:
population

,Name,Pop,State,abbr
0,Mississippi,2976149,28,MS
1,Missouri,6137428,29,MO
2,Montana,1068778,30,MT
3,Nebraska,1934408,31,NE
4,Nevada,3080156,32,NV
5,New Hampshire,1359711,33,NH
6,New Jersey,8882190,34,NJ
7,New Mexico,2096829,35,NM
8,New York,19453561,36,NY
9,North Carolina,10488084,37,NC


In [84]:
#response = requests.get('https://chronicdata.cdc.gov/resource/hn4x-zwk7.json?question=Percent of adults aged 18 years and older who have an overweight classification&yearstart=2021')
endpoint = 'https://chronicdata.cdc.gov/resource/hn4x-zwk7.json?question=Percent of adults who engage in no leisure-time physical activity&yearstart=2021'
query = {'$limit':89000,'$offset':0,'$$app_token': 'mG6piKQ1eLQSagm22PyMvQj3n'}

results = []

while True:
    response = requests.get(endpoint,params = query)
    if len(response.json()) == 0:
        break
    results.extend(response.json())
    
    query['$offset'] += len(response.json())


In [85]:
leisure = pd.DataFrame(results)

In [86]:
leisure

,yearstart,yearend,locationabbr,locationdesc,datasource,class,topic,question,data_value_type,data_value,...,stratification1,stratificationcategoryid1,stratificationid1,income,age_years,gender,data_value_footnote_symbol,data_value_footnote,race_ethnicity,total
0,2021,2021,WA,Washington,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,Value,17.2,...,Some college or technical school,EDU,EDUCOTEC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021,2021,UT,Utah,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,Value,25.4,...,"$25,000 - $34,999",INC,INC2535,"$25,000 - $34,999",NaN,NaN,NaN,NaN,NaN,NaN
2,2021,2021,WA,Washington,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,Value,37.0,...,Less than high school,EDU,EDUHS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,2021,NE,Nebraska,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,Value,12.6,...,College graduate,EDU,EDUCOGRAD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,2021,WI,Wisconsin,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,Value,35.0,...,Less than high school,EDU,EDUHS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1535,2021,2021,NH,New Hampshire,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,Value,21.5,...,Female,GEN,FEMALE,NaN,NaN,Female,NaN,NaN,NaN,NaN
1536,2021,2021,NJ,New Jersey,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,Value,34.1,...,Hispanic,RACE,RACEHIS,NaN,NaN,NaN,NaN,NaN,Hispanic,NaN
1537,2021,2021,PR,Puerto Rico,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,Value,37.4,...,"$35,000 - $49,999",INC,INC3550,"$35,000 - $49,999",NaN,NaN,NaN,NaN,NaN,NaN
1538,2021,2021,WI,Wisconsin,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,Value,11.8,...,"$75,000 or greater",INC,INC75PLUS,"$75,000 or greater",NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
df = df.merge(population[['abbr','State']],left_on = 'state',right_on = 'abbr').rename(columns = {'State':'State_ID_fk'})

In [69]:
df

,cc,state,city,zip,a1,abbr,State_ID_fk
0,US,RI,Pawtucket,02860,50 Ann Mary St,RI,44
1,US,RI,Warwick,02888,1222 Warwick Ave,RI,44
2,US,RI,Woonsocket,02895,1500 Diamond Hill Rd,RI,44
3,US,RI,East Providence,02914,585 Taunton Ave,RI,44
4,US,RI,Cumberland,02864,1800 Mendon Rd,RI,44
...,...,...,...,...,...,...,...
2369,US,MT,Helena,59601,3035 N Sanders St,MT,30
2370,US,MT,Missoula,59808,2640 N Reserve St,MT,30
2371,US,MT,Great Falls,59405,726 10th Ave S,MT,30
2372,US,MT,Billings,59102,640 S 24th Street W,MT,30


In [95]:
health = health.merge(population[['abbr','State']],left_on = 'locationabbr',right_on = 'abbr').rename(columns = {'State':'State_ID_fk'})




In [87]:
leisure = leisure.merge(population[['abbr','State']],left_on = 'locationabbr',right_on = 'abbr').rename(columns = {'State':'State_ID_fk'})



In [88]:
leisure = leisure.drop(columns = 'geolocation')

In [99]:
# df health leisure population
df.to_sql(name = 'gym_locations', con = conn, if_exists = 'replace')

2374

In [96]:
health.to_sql(name = 'weight_condition', con = conn, if_exists = 'replace')

1456

In [91]:
health.columns

Index(['yearstart', 'yearend', 'locationabbr', 'locationdesc', 'datasource',
       'class', 'topic', 'question', 'data_value_type', 'data_value',
       'data_value_alt', 'low_confidence_limit', 'high_confidence_limit',
       'sample_size', 'age_years', 'classid', 'topicid', 'questionid',
       'datavaluetypeid', 'locationid', 'stratificationcategory1',
       'stratification1', 'stratificationcategoryid1', 'stratificationid1',
       'education', 'income', 'race_ethnicity', 'gender',
       'data_value_footnote_symbol', 'data_value_footnote', 'total', 'abbr_x',
       'State_ID_fk', 'abbr_y', 'State_ID_fk'],
      dtype='object')

In [97]:
leisure.to_sql(name = 'leisure', con = conn, if_exists = 'replace')

1456

In [109]:
population = population.rename(columns = {'State': 'State_ID_fk'})

In [110]:
population.to_sql(name = 'states', con = conn, if_exists = 'replace')

52